In [65]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from data import *
from _version import __version__
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:

from _version import __version__
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()
    # delete incomplete experiments
    dbHandler.delete_incomplete_experiments()
print("Current code version, ", __version__)

Current code version,  12


In [67]:
# get the completed experiments with the current code version ordered by date
df_experiments = df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]
df_experiments = df_experiments.sort_values(by="date",ascending=False)
df_experiments.head(2)

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,tau,d,erm_methods,completed
10,fdc88c4c-5898-48aa-9fcb-1e9f0f7b9ab9,Default Experiment,35.551177,12,2023-03-30 13:12:48,10,10,"[0.2, 1.0, 2.0, 5.0]",[0],[1],0.0,1000,"[""gd""]",1


In [68]:
# extract and print the top experiment_id
experiment_id = df_experiments.iloc[0]["experiment_id"]
print(experiment_id)

fdc88c4c-5898-48aa-9fcb-1e9f0f7b9ab9


In [69]:
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == experiment_id]
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
gd = erm[erm["chosen_minimizer"] == "gd"]
# set alpha, epsilon, tau, lam as index in gd and state_evolution
gd = gd.set_index(["alpha","epsilon","tau","lam"])
state_evolution = state_evolution.set_index(["alpha","epsilon","tau","lam"])
# from state_evolution keep generalization_error, sigma, q, m, cosb
state_evolution = state_evolution[["generalization_error","sigma","q","m","cosb"]]
# add a suffix _state_evolution to the columns
state_evolution.columns = [col+"_state_evolution" for col in state_evolution.columns]
# from gd keep generalization_error_erm	generalization_error_overlap Q	rho	m	cosb
gd = gd[["generalization_error_erm","generalization_error_overlap","Q","rho","m","cosb"]]
# add a suffix _erm to the columns
gd.columns = [col+"_erm" for col in gd.columns]
# join the two dataframes
df = state_evolution.join(gd)
# reorder the columns
df = df[['generalization_error_state_evolution', 'generalization_error_erm_erm', 'generalization_error_overlap_erm', 'sigma_state_evolution', 'q_state_evolution','Q_erm','m_state_evolution','m_erm','cosb_state_evolution','cosb_erm', 'rho_erm']]
# average all quantities over the index and obtain also the standard deviation
df = df.groupby(level=[0,1,2,3]).agg([np.mean,np.std])
# display all columns
pd.set_option('display.max_columns', None)
df

generalization_error_state_evolution       \
                                                      mean  std   
alpha epsilon tau lam                                             
0.2   0.0     0.0 1.0                             0.480177  0.0   
1.0   0.0     0.0 1.0                             0.411330  0.0   
2.0   0.0     0.0 1.0                             0.345630  0.0   
5.0   0.0     0.0 1.0                             0.232751  0.0   

                      generalization_error_erm_erm            \
                                              mean       std   
alpha epsilon tau lam                                          
0.2   0.0     0.0 1.0                     0.387493  0.004041   
1.0   0.0     0.0 1.0                     0.272885  0.005620   
2.0   0.0     0.0 1.0                     0.203870  0.005145   
5.0   0.0     0.0 1.0                     0.124418  0.002345   

                      generalization_error_overlap_erm            \
                                                  mean       std   
alpha epsilon tau lam                                              
0.2   0.0     0.0 1.0                         0.387901  0.004326   
1.0   0.0     0.0 1.0                         0.272377  0.005614   
2.0   0.0     0.0 1.0                         0.203561  0.004165   
5.0   0.0     0.0 1.0                         0.124713  0.002505   

                      sigma_state_evolution      q_state_evolution       \
                                       mean  std              mean  std   
alpha epsilon tau lam                                                     
0.2   0.0     0.0 1.0              0.961896  0.0          0.032908  0.0   
1.0   0.0     0.0 1.0              0.832369  0.0          0.175880  0.0   
2.0   0.0     0.0 1.0              0.711782  0.0          0.367657  0.0   
5.0   0.0     0.0 1.0              0.499874  0.0          0.927348  0.0   

                          Q_erm           m_state_evolution          m_erm  \
                           mean       std              mean  std      mean   
alpha epsilon tau lam                                                        
0.2   0.0     0.0 1.0  0.031349  0.000770          0.062074  0.0  0.061036   
1.0   0.0     0.0 1.0  0.179206  0.002687          0.276109  0.0  0.279605   
2.0   0.0     0.0 1.0  0.381521  0.003498          0.484342  0.0  0.494637   
5.0   0.0     0.0 1.0  1.022047  0.005041          0.889271  0.0  0.941993   

                                cosb_state_evolution       cosb_erm            \
                            std                 mean  std      mean       std   
alpha epsilon tau lam                                                           
0.2   0.0     0.0 1.0  0.002589             0.062236  0.0  0.344904  0.012745   
1.0   0.0     0.0 1.0  0.008651             0.274977  0.0  0.655592  0.013336   
2.0   0.0     0.0 1.0  0.013565             0.466179  0.0  0.802324  0.007854   
5.0   0.0     0.0 1.0  0.027277             0.744369  0.0  0.924196  0.003006   

                        rho_erm            
                           mean       std  
alpha epsilon tau lam                      
0.2   0.0     0.0 1.0  0.999546  0.039793  
1.0   0.0     0.0 1.0  1.016085  0.056000  
2.0   0.0     0.0 1.0  0.996548  0.043275  
5.0   0.0     0.0 1.0  1.017268  0.057346

In [ ]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)